<a href="https://colab.research.google.com/github/AmirGhnbr/Big-Food-Vision/blob/dev_branch/Big_Food_Vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Alongside attempting to beat the DeepFood paper, we're going to use three two methods to significantly improve the speed of our model training:

1. Prefetching
2. Mixed precision training
3. Using TPUs and large batch size to speed up training trying to replicate [[Ying et el 2018](https://arxiv.org/abs/1811.06992)]